Markdown text using HTML

In [1]:
#version history
#0 - basic read data and plot
#1 - incorporate widgets/buttons
#2 - load data individually, legend
#3 - clear plot
#4 - rename files

In [2]:
#import libraries
import pandas as pd
import os
import numpy as np
import pyqtgraph as pg
import pyqtgraph.console
import random
import time
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.dockarea import *

In [3]:
#define functions

def load(self):
    global state
    global loaded
    #prompt for data file
    fileName = QtGui.QFileDialog.getOpenFileName(None, 'Choose File', "*.csv")
    
    
    
    #read csv file
    data = pd.read_csv(fileName[0], low_memory=False,header=1)
    print(data)
    #drop last row which is full of NaN
    data.drop(data.tail(1).index, inplace = True)
    

    #find all sensorid's
    IDs = data.SensorID.unique()

    for val in IDs:
        #testing
        
        #keep only if matches sensor
        IDData = data[data.SensorID == val]

        #manipulate data from csv

       #iloc [row:row, col:col]
        #grab date and time values
        date = IDData.iloc[:,0]
        time = IDData.iloc[:,1]
        dateTime = date + " " + time
        #print(dateTime)

        #o2 unit conversion to O2 saturation.
        unit=20.95/100

        #don't do anything if already in saturation units
        if IDData.iloc[0,9] != "%O2":
            unit = 1 

        #grab o2 values
        o2 = IDData.iloc[:,8] / unit
        #print(o2)
        #convert pandas type array to numpy array for graphing
        x = dateTime
        y = o2

        #must convert to datetime64
        #o2 values are already in float64
        x = x.astype('datetime64')

        #calculate deltaT and convert to float64 type to allow graphing
        x = x[:] - x.iloc[0]
        xsec = x/np.timedelta64(1, 's')
        xhr = xsec/60/60

        #sort data
        zipped_data = zip(xhr,y)
        sorted_data = sorted(zipped_data, key = lambda x: x[0])

        #unzip
        x, y = zip(*sorted_data)
        
        #change color random
        coloura = random.randint(0,255)
        colourb = random.randint(0,255)
        colourc = random.randint(0,255)

        #plot
        curve = w3.plot(x,y, pen = pg.mkPen(color=(coloura,colourb,colourc)))
        
        #add filename to table w2
        loaded = np.append(loaded, fileName[0])
        w2.setData(loaded)
        
        #add just filename to legend
        base = os.path.basename(fileName[0])
        name = os.path.splitext(base)[0]
        legend.addItem(curve, name)
        
#clear plot, legend and table
def clear(self):
    global loaded  
    w3.clear()
    legend.clear()
    loaded = np.array([], dtype=object)
    w2.setData(loaded)
        

    

In [4]:
#create window and docks
app = QtGui.QApplication([])
win = QtGui.QMainWindow()
area = DockArea()
win.setCentralWidget(area)
win.resize(1500,1500)
win.setWindowTitle('DODA')

#State Docks
d1 = Dock("Dock1", size =(200,1)) #width, height; 1 = minimum
d2 = Dock("Dock2",size = (1500, 500)) #table
d3 = Dock("Dock3", size =(1500,1500)) #graph

#Dock Locations
area.addDock(d1,'bottom') #buttons
area.addDock(d3,'top') #graph
area.addDock(d2,'right', d1) #table


#set up docks
#dock 1: choose data to load and set axis
#state objects
w1 = pg.LayoutWidget()
loadBtn = QtGui.QPushButton('Load Data')
#updateAxis = QtGui.QPushButton('Update Axis')
clearBtn = QtGui.QPushButton('Clear Plot')


#organize objects
w1.addWidget(loadBtn, row=0, col=0)
w1.addWidget(clearBtn, row=1, col=0)

d1.addWidget(w1)
loadBtn.clicked.connect(load)
clearBtn.clicked.connect(clear)


#dock 2: list data included
w2 = pg.TableWidget() #row, col
loaded = np.array([], dtype=object)
w2.setData(loaded)
#w2.setBackground('r')
#w2.setHorizontalHeaderLabels(["Loaded Data"])
#w2.setColumnWidth(0, 950)
d2.addWidget(w2)


#dock 3: graph
labelStyle = {'color': '#FFF', 'font-size': '20px'}
w3 = pg.PlotWidget(title = 'O2 Saturation [%] vs Time [hrs]', **labelStyle)
w3.showGrid(x=True, y=True)
w3.setLabel('left', 'O2 Saturation', units='%', **labelStyle)
w3.setLabel('bottom', 'Time', units='Hrs', **labelStyle)

font=QtGui.QFont()
font.setPixelSize(18)
w3.getAxis("bottom").tickFont = font
legend = pg.LegendItem((80,60), offset=(100,40), labelTextSize='12pt') #White Background

legend.setParentItem(w3.graphicsItem())

d3.addWidget(w3)


#load up window
win.show()

## Start Qt event loop unless running in interactive mode or using pyside.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()


             Date      Time  Channel           User   SensorID   Sensor_Name  \
0      01/18/2021  11:03:10        1        default  484648071  3/8 FTC - 2"   
1      01/18/2021  11:03:15        1        default  720215605  3/8 FTC - 1"   
2      01/18/2021  11:03:20        1        default  484648071  3/8 FTC - 2"   
3      01/18/2021  11:03:25        1        default  720215605  3/8 FTC - 1"   
4      01/18/2021  11:03:30        1        default  484648071  3/8 FTC - 2"   
...           ...       ...      ...            ...        ...           ...   
15116  01/19/2021  08:02:50        1        default  484648071  3/8 FTC - 2"   
15117  01/19/2021  08:02:55        1        default  720215605  3/8 FTC - 1"   
15118  01/19/2021  08:03:00        1        default  484648071  3/8 FTC - 2"   
15119  01/19/2021  08:03:05        1        default  720215605  3/8 FTC - 1"   
15120        Date      Time  delta_t  Device_Serial    Channel          User   

          delta_t Time_Unit    Value O2